### Import Cells

It's recomended that every import statement is declared in the next cell.

Just remember to be careful when using <font color=green>*"from __lib__ import __(function, class, ...)__"*</font> statements, these can override other libraries functions and classes.

In [1]:
# Libraries imports
import time
from pathlib import Path

import ipywidgets as widgets
import IPython.display

import py4syn
from py4syn.epics.MotorClass import Motor
from epics import PV

import plotly.offline as py
import plotly.graph_objs as go
import plotly.figure_factory as FF
import plotly.io as pio
from plotly import tools

import numpy as np
import pandas as pd
import uuid

# Enable offline plots from plotly
py.init_notebook_mode(connected=True)

### Helper Functions and Classes

The next cells defines helpers functions and classes for the notebook. These __should not be edited unless__ the user feels confortable in "tweking" these functions for its own use. To edit these cells, the cell's metadata value "editable" needs to be true.

<sup><sup>Any bug or unexpected behavior from any of these functions and classes, please contact SOL.</sup></sup>

In [2]:
# Logging function that write log information in a file and also in the notebook output cells (if this config is set True)
def logprint(string, mode="[INFO]", config={'log_cell': widgets.Checkbox(value=False)}):
    # Log information has a time stamp using GMT-0 time to avoid local computer time problems
    ts = time.gmtime()
    
    # Logs will be stored in a .log directory that will be created if it doesn't exist
    year_month_day = time.strftime("%Y-%m-%d", ts)
    file_name = Path('.logs/' + year_month_day + '-log.txt')

    if not file_name.parent.is_dir():
        file_name.parent.mkdir()
    
    time_stamp = time.strftime("%Y-%m-%d %H:%M:%S", ts)
    
    # Write log in the file
    with open(str(file_name), "a") as f:
        f.write(time_stamp + ' | ' + mode + ' ' + string + '\n')
    
    # Write log in stdout
    if config['log_cell'].value:
        print(time_stamp, string)
    

In [3]:
# Util function to create a motor with logging informations
def configurate_motor(motor_pv_name='', motor_name='', config={'log_cell': widgets.Checkbox(value=False)}):
    motor = None
    
    try:
        motor = Motor(motor_pv_name, motor_name)
        logprint("Motor: \'" + motor_pv_name + "\' succesfully created as \'" + motor_name + "\'", config=config)
    except Exception as e:
        logprint("Error trying to create Motor: \'" + motor_pv_name + "\' as \'" + motor_name + "\'", "[ERROR]", config=config)
        logprint(str(e), "[ERROR]", config=config)
        
    return motor

In [4]:
# Default Layouts for both Bounded Float Text and Buttons
_default_bounded_text_layout = {
    'disabled': False,
}

_default_button_layout = {
    'description': 'Set Target VAL',
    'disabled': False,
    'button_style': 'success', # 'success', 'info', 'warning', 'danger' or ''
    'tooltip': 'Click me',
    'icon': ''
}

In [5]:
# Class MotorSetValueButton encapsulates a bounded float text with a button to a provided motor
# This class connects the button to the bouded float value and set the motor position
class MotorSetValueButton(widgets.Button):
    
    def __init__(self, motor, config, *args, **kwargs):
        widgets.Button.__init__(self, *args, **kwargs)
        
        # Config
        self.config = config
        
        # Motor associated to the button
        self.motor = motor
        if type(motor) != py4syn.epics.MotorClass.Motor:
            logprint("Passed to class MotorSetValueButton constructor a argument with wrong type. Expected py4syn.epics.MotorClass.Motor, received " + str(type(motor)), config=self.config)
            
            raise("Passed to class MotorSetValueButton constructor a argument with wrong type. Expected py4syn.epics.MotorClass.Motor, received " + str(type(motor)))
        
        # Bounded float text associated to the button
        self.bounded_float = widgets.BoundedFloatText(
                                value=motor.getRealPosition(),
                                min=motor.getLowLimitValue(),
                                max=motor.getHighLimitValue(),
                                step=0.01,
                                description=motor.pvName,
                                disabled=False
                              )
        
        # class Button values for MotorSetValueButton
        self.description = 'Set Target VAL'
        self.disabled = False
        self.button_style = 'success'
        self.tooltip = 'Click me'
        self.icon = ''    
        
        # Set callback function for click event
        self.on_click(self._motor_set_val_button)
        
        # Widgets Boxes
        self.output = widgets.Output()
        
    @staticmethod
    def _motor_set_val_button(b):
        # Clear previous logs outputs
        b.output.clear_output()
        
        # with statement to output logs in stdou (if this option is enabled)
        with b.output:
            # Change button to a "clicked status"
            b.disabled = True
            b.button_style = ''

            logprint("Starting motor " + b.motor.pvName + " movement to absolute value " + str(b.bounded_float.value), config=b.config)
            try:
                # Move the motor to target absolute position
                b.motor.setAbsolutePosition(b.bounded_float.value, waitComplete=False)
                logprint("Set absolute value " + str(b.bounded_float.value) + " for motor " + b.motor.pvName, config=b.config)
            except Exception as e:
                # If any error occurs, log that but dont stop code exection
                logprint("Error in setting absolute value " + str(b.bounded_float.value) + " for motor " + b.motor.pvName, "[ERROR]", config=b.config)
                logprint(str(e), "[ERROR]", config=b.config)

            # Change button layout back to normal
            b.disabled = False
            b.button_style = 'success'
        

    def box_motor_button(self):
        return widgets.HBox([self.bounded_float, self])    
    
    
    def box_motor_output(self):
        return self.output 
    
    
    def display_motor_button(self):
        display(self.box_motor_button(), self.output)


In [6]:
class StartMotorsButton(widgets.Button):
    
    def __init__(self, config, *args, **kwargs):
        widgets.Button.__init__(self, *args, **kwargs)
        
        # Config
        self.config = config
        
        # Text box to write the motors
        self.text = widgets.Textarea(
            value='',
            placeholder='Example: IOC:m1 IOC:m3 LNLS:TEST:motor_g3',
            description='',
            disabled=False
        )
        
        # class Button values for StartMotorsButton
        self.description='Start Motor Initializaton'
        self.disabled=False
        self.button_style='success'
        self.tooltip='Click me'
        self.icon=''
        self.layout = widgets.Layout(width='300px')
        
        # Boxes
        self.motors_value_buttons = []
        
        # Motors
        self.motors_list = []
        
        # Set callback function for click event
        self.on_click(self._start_button)
        
        # Widgets displays
        self.start_button = widgets.VBox([self.text, self])
        self.target_value_buttons = widgets.VBox([widgets.Label("No motors initilized to be showed.")])
        
        # Logging
        self.output = widgets.Output()
        
        
    @staticmethod
    def _start_button(b):
        # Clear previous logs outputs
        b.output.clear_output()
        
        # with statement to output logs in stdou (if this option is enabled)
        with b.output:
            # Change button to a "clicked status"
            b.disabled = True
            b.button_style = ''
            b.description='Initializing...'
            # We should sleep for some time to give some responsiveness to the user
            time.sleep(0.5)

            # Get motors PV names from the text box
            motor_list_names = b.text.value.split(' ')

            logprint("Starting motors " + ', '.join(motor_list_names) + " initialization", config=b.config)

            # Create motors and add a MotorSetValueButton to them
            try:
                b.motors_list = [configurate_motor(name, ' '.join(name.split(':')[-2:]), b.config) for name in motor_list_names]

                b.motors_value_buttons = [MotorSetValueButton(motor, b.config) for motor in b.motors_list]

                # Set Target Value Button Widget children to all initilized motors boxes
                widgets_boxes = [motor_button.box_motor_button() for motor_button in b.motors_value_buttons]
                widgets_target_outputs = [motor_button.box_motor_output() for motor_button in b.motors_value_buttons]
                b.target_value_buttons.children = tuple(widgets_boxes + widgets_target_outputs)

                logprint("Finished motors " + ', '.join(motor_list_names) + " initialization", config=b.config)              
            except Exception as e:
                # If any error occurs, log that but dont stop code exection
                logprint("Error in initialziation of motors " + ', '.join(motor_list_names), "[ERROR]", config=b.config)
                logprint(str(e), "[ERROR]", config=b.config)

                # Reset Target Value Boxes to default Label
                b.target_value_buttons.children = (widgets.Label("No motors initilized to be showed."),)

            # Change button layout back to normal
            b.disabled = False
            b.button_style = 'success'
            b.description='Start Motor Initializaton'   
    
    
    def display_start_button(self):
        display(self.start_button, self.output)
    
    
    def display_motors_targ_buttons(self):
        display(self.target_value_buttons)


In [7]:
class MotorsMonitor(widgets.Button):
    
    def __init__(self, config, *args, **kwargs):
        widgets.Button.__init__(self, *args, **kwargs)
        
        # Config
        self.config = config
        
        # Text box to write the motors
        self.text = widgets.Textarea(
            value='',
            placeholder='Example: IOC:m1 IOC:m3 LNLS:TEST:motor_g3',
            description='',
            disabled=False
        )
        
        # class Button values for MotorsMonitor
        self.description='Start Motor Monitoring'
        self.disabled=False
        self.button_style='success'
        self.tooltip='Click me'
        self.icon=''
        self.layout = widgets.Layout(width='300px')
        
        # Boxes
        self.motors_values = {}
        
        # Motors
        self.motors_list = []
        
        # Set callback function for click event
        self.monitoring_status = False
        self.on_click(self._monitor_button)
        
        # Main widget
        self.main_box = widgets.VBox([self, self.text])
        self.output = widgets.Output()
        
    @staticmethod
    def _monitor_button(b):
        # Clear previous logs outputs
        b.output.clear_output()
        
        # with statement to output logs in stdou (if this option is enabled)
        with b.output:
            # Disabel button to avoid double commands
            b.disabled=True

            if b.monitoring_status:
                # Change button appearence
                b.description='Start Motor Monitoring'
                b.button_style='success'

                # Stop displaying the motors widgets, show only button and text box
                b.main_box.children = (b.main_box.children[0], b.text,)

                logprint("Stopped monitoring motors " + ', '.join([motor.pvName for motor in b.motors_list]), config=b.config)

                # Reset motor list
                b.motors_list = []      
            else:
                # Change button appearence
                b.description='Stop Motor Monitoring'
                b.button_style = 'danger'

                # Stop displaying text box
                b.main_box.children = (b.main_box.children[0],)

                # Get motors PV names from the text box
                motor_list_names = b.text.value.split(' ')
                logprint("Started monitoring motors " + ', '.join(motor_list_names), config=b.config)

                # Create motors and add a monitor callback to them
                # Also add these motor values as children of main_box widget
                try:
                    b.motors_list = [configurate_motor(name, ''.join(name.split(':')[-2:])) for name in motor_list_names]

                    for motor in b.motors_list:
                        index = motor.motor.add_callback('RBV', b._monitor_callback)

                        rbv = motor.getRealPosition()
                        b.motors_values[motor.pvName + ".RBV"] = (rbv, str(rbv))

                        b.main_box.children += (widgets.HBox([widgets.Label(motor.pvName + ".RBV"), widgets.Label(str(rbv))]),)

                    logprint("Monitoring motors " + ', '.join(motor_list_names), config=b.config)        

                except Exception as e:
                    # If any error occurs, log that but dont stop code exection
                    logprint("Error in monitoring motors " + ', '.join(motor_list_names), "[ERROR]", config=b.config)
                    logprint(str(e), "[ERROR]", config=b.config)

            # Re enable button
            b.disabled=False

            # Switch status
            b.monitoring_status = not b.monitoring_status
               

    def _monitor_callback(self, pvname='', value=0, char_value='', **kw):
        self.motors_values[pvname] = (value, char_value)
        
        if monitoring_status:
            for widget in self.main_box.children[1:]:
                if widget.children[0].value == pvname:
                    widget.children[1].value = char_value


    def display_monitor_motors(self):
        display(self.main_box, self.output)


In [8]:
class PVMonitor(widgets.Button):
    
    def __init__(self, config, *args, **kwargs):
        widgets.Button.__init__(self, *args, **kwargs)
        
        # Config
        self.config = config
        
        # Text box to write the motors
        self.text = widgets.Textarea(
            value='',
            placeholder='Example: IOC:m1.DMOV IOC:m3.RBV LNLS:ANEL:corrente.VAL',
            description='',
            disabled=False
        )
        
        # class Button values for PVMonitor
        self.description='Start PV Monitoring'
        self.disabled=False
        self.button_style='success'
        self.tooltip='Click me'
        self.icon=''
        self.layout = widgets.Layout(width='300px')
        
        # Boxes
        self.pv_values = {}
        
        # PVs
        self.pv_list = []
        
        # Set callback function for click event
        self.monitoring_status = False
        self.on_click(self._monitor_button)
        
        # Main widget
        self.main_box = widgets.VBox([self, self.text])
        self.output = widgets.Output()
        
    @staticmethod
    def _monitor_button(b):
        # Clear previous logs outputs
        b.output.clear_output()
        
        # with statement to output logs in stdou (if this option is enabled)
        with b.output:
            # Disabel button to avoid double commands
            b.disabled=True

            if b.monitoring_status:
                # Change button appearence
                b.description='Start PV Monitoring'
                b.button_style='success'

                # Stop displaying the motors widgets, show only button and text box
                b.main_box.children = (b.main_box.children[0], b.text,)

                logprint("Stopped monitoring PVs " + ', '.join([pv.pvname for pv in b.pv_list]), config=b.config)

                # Reset motor list
                b.pv_list = []      
            else:
                # Change button appearence
                b.description='Stop PV Monitoring'
                b.button_style = 'danger'

                # Stop displaying text box
                b.main_box.children = (b.main_box.children[0],)

                # Get motors PV names from the text box
                pv_list_names = b.text.value.split(' ')
                logprint("Started monitoring PVs " + ', '.join(pv_list_names), config=b.config)

                # Create PVs and add a monitor callback to them
                # Also add these PVs values as children of main_box widget
                try:
                    b.pv_list = [PV(name) for name in pv_list_names]

                    for pv in b.pv_list:
                        pv.add_callback(b._monitor_callback)

                        value = pv.get()
                        b.pv_values[pv.pvname] = (value, str(value))

                        b.main_box.children += (widgets.HBox([widgets.Label(pv.pvname), widgets.Label(str(value))]),)

                    logprint("Monitoring PVs " + ', '.join(pv_list_names), config=b.config)        

                except Exception as e:
                    # If any error occurs, log that but dont stop code exection
                    logprint("Error in monitoring PVs " + ', '.join(pv_list_names), "[ERROR]", config=b.config)
                    logprint(str(e), "[ERROR]", config=b.config)

            # Re enable button
            b.disabled=False

            # Switch status
            b.monitoring_status = not b.monitoring_status
               

    def _monitor_callback(self, pvname='', value=0, char_value='', **kw):
        self.pv_values[pvname] = (value, "{:.3f}".format(value))
        
        for widget in self.main_box.children[1:]:
            if widget.children[0].value == pvname:
                widget.children[1].value = "{:.3f}".format(value)


    def display_monitor_pvs(self):
        display(self.main_box, self.output)


In [9]:
import py4syn.utils.scan as sc

class ScanPlot():
    def __init__(self, name, *args, **kwargs):
        # Scan figure widget that will be displayed
        self.figure = go.FigureWidget()
      

    def add_scatter(self, initial_x, initial_y, name, mode='lines+markers'):
        self.figure.add_traces([go.Scatter(x=initial_x,
                                          y=initial_y,
                                          mode=mode,
                                          name=name)])
        
        
    def plot(self, x, y, label):
        for trace in self.figure['data']:
            if trace['name'] == label:
                trace['x'] = x
                trace['y'] = y
    
    
    def display(self):
        return self.figure


In [10]:
import argparse

def parseCommandLine():
    parser = argparse.ArgumentParser(description="Perform a scan with \
    specified devices (e.g.: motors) and the list of counters provided by the \
    configuration file.")

    parser.add_argument('-l', '--list-configurations',
                        help='list configurations instead of scanning',
                        action='store_true')

    parser.add_argument('-c', '--configuration',
                        help='choose a counter configuration file',
                        default='default')

    parser.add_argument('--optimum',
                        help='move motor to the optimal point according to \
                        this counter after scan')

    parser.add_argument('--repeat',
                        help='scan multiple times',
                        type=int,
                        default=1)

    parser.add_argument('--sleep',
                        help='sleep time before each acquisition',
                        type=float,
                        default=0)

    parser.add_argument('-m', '--message',
                        help='string of comments to put in output file header')

    parser.add_argument('-o', '--output',
                        help='output data to file output-prefix/<fileprefix>_nnnn')

    parser.add_argument('-s', '--sync',
                        help='write to the output file after each point',
                        action='store_true')

    parser.add_argument('--snake',
                        help='snake scan mode',
                        action='store_true')

    parser.add_argument('--motor',
                        help='list of motors',
                        nargs='+')

    parser.add_argument('--xlabel',
                        help='motor which position is shown in x axis \
                        (if not set, point index is shown instead)',
                        default='points')

    # Arguments describing a run
    group_run = parser.add_argument_group(
                        description="Arguments for describing a given run \
                        (define this set of arguments again for each new run):")

    group_run.add_argument('--start',
                        help='list of start positions of each device',
                        nargs='+',
                        action='append',
                        type=float)

    group_run.add_argument('--end',
                        help='list of end positions of each device',
                        nargs='+',
                        action='append',
                        type=float)

    group_run.add_argument('--step-or-points',
                        help='list of step size (or number of points) for each device',
                        nargs='+',
                        action='append',
                        type=float)

    group_run.add_argument('--time',
                        help='acquisition time',
                        nargs=1,
                        action='append',
                        type=float)

    args = parser.parse_args()
    if args.list_configurations:
        listConfigurations()
        raise SystemExit(1)

    # Just for args name compatibility
    args.stepOrPoints = args.step_or_points
    del(args.step_or_points)
    del(args.list_configurations)

    args_dict = vars(args)
    return args_dict


In [11]:
from scan_utils.scan import ScanOperationCLI
from py4syn.utils import scan as scanModule

from scan_utils import cleanup, die

class JupyScan(ScanOperationCLI):
    def __init__(self, motor, start, end, stepOrPoints, time, configuration='default',
            optimum=False, sync=False, output=None, message=None, repeat=1, sleep=0,
            waitPlotter=True, plotXFactor=1, snake=False, xlabel='points'):
        
        super().__init__(motor, start, end, stepOrPoints, time, configuration,
            optimum, sync, output, message, repeat, sleep,
            waitPlotter, plotXFactor, snake, xlabel)
        
        
    def plot(self, plotter, scan, pos, idx):
        data = scanModule.getScanData()
        user = scanModule.getUserDefinedDataFields()
        position = data[self.xlabel][-1]

        for counter in self.configuration.runtime:
            c = self.configuration['counters'][counter]
            
            label = c['label']
            if c.get('plot', True):
                plotter.plot(data[self.xlabel], data[label], label)
            elif counter in user:
                data[label].append(processUserField(c))

    # Override default plot procedure
    def configurePlot(self):
        from py4syn.utils.plotter import Plotter
        scanModule.setPlotGraph(False)

        if self.configuration.somePlot() :
            p = ScanPlott(self.output or 'Scan')
            
            for counter in self.configuration.runtime:
                c = self.configuration['counters'][counter]
                label = c['label']
                
                p.add_scatter([0], [0], label)
                
            
            scanModule.setPostOperationCallback(lambda **kw: self.plot(p, **kw))
        else:
            p = None

        return p


    def run(self):
        self.onOperationBegin()
        plotter = self.configurePlot()

        # Display plot using IPython display widget
        IPython.display.display(plotter.display())

#         if plotter is not None:
#             axes = self.generateAxes(plotter)

        # Build arguments for scan method
        scan_args = []
        for i in range(len(self.motor)):
            scan_args.append(self.motor[i])
            scan_args.append(self.points[i])
        scan_args.append(len(self.points[0]))
        scan_args.append(self.times)
        scan_args = tuple(scan_args)

        print('\nEstimated time: ' + str(self.getEstimatedTime(self.times)) + '\n')

        for i in range(self.repeat):
            self.onScanBegin()

#             if plotter is not None:
#                 next(axes)

            try:
                scanModule.scan(*scan_args)
            except Exception as e:
                raise RuntimeError('Error executing scan: ' + str(e))

            self.onScanEnd()

            self.fitValues()

        if self.optimum:
            self.goToOptimum()

        cleanup()
        self.onOperationEnd()
#         if self.waitPlotter:
#             plotter.plot_process.join()


    def runScan():
        args = parseCommandLine()

        try:
            scan = ScanOperationCLI(**args)

            scan.run()
        except (OSError, RuntimeError) as e:
            die(e)


In [1]:
class ScanButton(widgets.Button):
    
    def __init__(self, config, *args, **kwargs):
        widgets.Button.__init__(self, *args, **kwargs)
        
        # Config
        self.config = config
        
        # Text box to write the motors to move
        self.text_motors = widgets.Textarea(
            value='',
            placeholder='Motors names from config.yml\nExample: solm3 gap2',
            description='',
            disabled=False
        )
        
        # Text box to write the configuration file
        self.text_config = widgets.Textarea(
            value='',
            placeholder='Name of the yml configuration file\nExample: default\n(PS: write "default" to load config.default.yml file)',
            description='',
            disabled=False
        )
        
        # Text box for start
        self.text_start = widgets.Text(
            value='',
            placeholder='Example for 2 motors: [1, 3], [7, 8]',
            description='',
            disabled=False
        )
        
        # Text box for end
        self.text_end = widgets.Text(
            value='',
            placeholder='Example for 2 motors: [2, 4], [8, 8.5]',
            description='',
            disabled=False
        )
        
        # Text box for step or points
        self.text_step_points = widgets.Text(
            value='',
            placeholder='Example for 2 motors: [1, 1], [0.5, 0.25]',
            description='',
            disabled=False
        )
        
        # Text box for time
        self.text_time = widgets.Text(
            value='',
            placeholder='Example for 2 motors: [1], [0.4]',
            description='',
            disabled=False
        )
        
        # Text box for output
        self.text_output = widgets.Text(
            value='',
            placeholder='Output file name, if left empty, file name will be the default name, "test"',
            description='',
            disabled=False
        )
        
        # Text box for optimum
        self.text_optimum = widgets.Text(
            value='',
            placeholder="Move motor to the optimal point according to \
                         this counter after scan. Leave empty for no move.",
            description='',
            disabled=True
        )
        
        # Optimum checkbox
        self.checkbox_optimum = widgets.Checkbox(
            value=False,
            description="",
            disabled=False,
            style={'description_width': 'initial'},
            layout = widgets.Layout(width='36px')
        )     

        self.checkbox_optimum.observe(self.change_checkbox_optimum, names=['value'])
        
        # class Button values for ScanButton
        self.description='Start Scan'
        self.disabled=False
        self.button_style='success'
        self.tooltip='Click me'
        self.icon=''
        self.layout = widgets.Layout(width='300px')
        
        # Boxes
        self.pv_values = {}
        
        # PVs
        self.motor_list = []
        
        # Set callback function for click event
        self.on_scan = False
        self.scan_ended = False
        self.config_loaded = False
        self.on_click(self._scan_button)
        
        # Main widget
        self.main_box = widgets.VBox([widgets.HBox([widgets.Label("Motors names", layout=widgets.Layout(width='150px')), self.text_motors]),
                                      widgets.HBox([widgets.Label("Configuration file name", layout=widgets.Layout(width='150px')), self.text_config]),
                                      widgets.HBox([widgets.Label("Start points", layout=widgets.Layout(width='150px')), self.text_start]),
                                      widgets.HBox([widgets.Label("End points", layout=widgets.Layout(width='150px')), self.text_end]),
                                      widgets.HBox([widgets.Label("Step or Points", layout=widgets.Layout(width='150px')), self.text_step_points]), 
                                      widgets.HBox([widgets.Label("Time", layout=widgets.Layout(width='150px')), self.text_time]),
                                      widgets.HBox([self.checkbox_optimum,
                                                    widgets.Label("Go to optimum of: ", layout=widgets.Layout(width='110px')),
                                                    self.text_optimum]),
                                      widgets.HBox([widgets.Label("Output file name", layout=widgets.Layout(width='150px')), self.text_output]),
                                      self])
        self.output = widgets.Output()

    
    def change_checkbox_optimum(self, change):
        self.text_optimum.disabled = not change.new
    
        
    @staticmethod
    def _scan_button(b):
        # Clear previous logs outputs
        b.output.clear_output()
        
        # with statement to output logs in stdout (if this option is enabled)
        with b.output:
            # Change button appearence
            b.description = 'Scanning'
            b.button_style = ''

            # Disable button to avoid double commands
            b.disabled = True

            # Disable box edition to avoid erros
            boxes = [b.text_motors, b.text_config, b.text_start, b.text_end, 
                    b.text_step_points, b.text_time, b.text_optimum, b.text_output]

            for box in boxes:
                box.disabled = True

            # Reset motor list
            b.motor_list = []

            # Get motors names from the text box
            motor_list_names = b.text_motors.value.split(' ')
            logprint("Scanning on motors" + ', '.join(motor_list_names), config=b.config)

            # Get config file name from the text box
            config_name = b.text_config.value
            logprint("YML config file: " + config_name, config=b.config)

            # Load scan parameters
            start = []
            end = []
            step_or_points = []
            time = []
            try:
                # Get lists from text boxes
                start = [eval(b.text_start.value)]
                end = [eval(b.text_end.value)]
                step_or_points = [eval(b.text_step_points.value)]
                time = [eval(b.text_time.value)]

                logprint("Loaded 'start, end, step or points, time' scan parameters", config=b.config)
            except Exception as e:
                # If any error occurs, log that but dont stop code exection
                logprint("Error loading 'start, end, step or points, time' scan parameters", "[ERROR]", config=b.config)
                logprint(str(e), "[ERROR]", config=b.config)

            # Get output file name
            output = b.text_output.value if b.text_output.value != '' else 'test'
            
            # Get absolute path to file, and create a scans directory if the directory doesn't exist
            mypath = Path().absolute() / 'scans'
            if not mypath.is_dir():
                mypath.mkdir()
            
            # Put the path to the output
            output = str(mypath) + '/' + output
            
            # Get optimum move option
#             optimum = b.text_optimum.value if b.checkbox_optimum.value else False
            optimum = False

            # Get sync option
            sync = False
            # Initiate scan
            try:
                js = JupyScan(motor_list_names, start, end, step_or_points, time, config_name, optimum, sync, output)
                
                js.run()
                logprint("Finished scan, output saved in file " + output, config=b.config)
            except Exception as e:
                # If any error occurs, log that but dont stop code exection
                logprint("Error in trying to scan", "[ERROR]", config=b.config)
                logprint(str(e), "[ERROR]", config=b.config)

            # Change button appearence
            b.description = 'Start Scan'
            b.button_style = 'success'

            # Re enable button
            b.disabled = False

            # Re enable box edition 
            for box in boxes:
                box.disabled = False
               

    def display_scan(self):
        display(self.main_box, self.output)


NameError: name 'widgets' is not defined

In [ ]:
class ScanParser():
    def __init__(self):
        self.parser = argparse.ArgumentParser(description="Perform a scan with \
                                        specified devices (e.g.: motors) and the list of counters provided by the \
                                        configuration file.")

        self.parser.add_argument('-l', '--list-configurations',
                            help='list configurations instead of scanning',
                            action='store_true')

        self.parser.add_argument('-c', '--configuration',
                            help='choose a counter configuration file',
                            default='default')

        self.parser.add_argument('--optimum',
                            help='move motor to the optimal point according to \
                            this counter after scan')

        self.parser.add_argument('--repeat',
                            help='scan multiple times',
                            type=int,
                            default=1)

        self.parser.add_argument('--sleep',
                            help='sleep time before each acquisition',
                            type=float,
                            default=0)

        self.parser.add_argument('-m', '--message',
                            help='string of comments to put in output file header')

        self.parser.add_argument('-o', '--output',
                            help='output data to file output-prefix/<fileprefix>_nnnn')

        self.parser.add_argument('-s', '--sync',
                            help='write to the output file after each point',
                            action='store_true')

        self.parser.add_argument('--snake',
                            help='snake scan mode',
                            action='store_true')

        self.parser.add_argument('--motor',
                            help='list of motors',
                            nargs='+')

        self.parser.add_argument('--xlabel',
                            help='motor which position is shown in x axis \
                            (if not set, point index is shown instead)',
                            default='points')

        # Arguments describing a run
        group_run = self.parser.add_argument_group(
                            description="Arguments for describing a given run \
                            (define this set of arguments again for each new run):")

        group_run.add_argument('--start',
                            help='list of start positions of each device',
                            nargs='+',
                            action='append',
                            type=float)

        group_run.add_argument('--end',
                            help='list of end positions of each device',
                            nargs='+',
                            action='append',
                            type=float)

        group_run.add_argument('--step-or-points',
                            help='list of step size (or number of points) for each device',
                            nargs='+',
                            action='append',
                            type=float)

        group_run.add_argument('--time',
                            help='acquisition time',
                            nargs=1,
                            action='append',
                            type=float)

        group_run.add_argument('-f',
                            help='ipython test',
                            nargs=1,
                            action='append',
                            type=str)
